In [1]:
!pip install packaging ninja
!pip install numpy torch
!pip install transformers torch
!pip install mamba-ssm --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.10.19:
      Successfully uninstalled nvidia-curand-cu12-10.3.10.19
  Attemp

In [2]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=3d0e4d125cb1da56cba9897c5899e99ee56d5eb35afb65dd1d25639292526b21
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=f66408ac3635a675401bff46965687fc35d1d8f394d710c29d1202d106f124f0
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [3]:
cd "/kaggle/input/ppp-2/pytorch/default/1/3DSS-Mamba-main/3DSS-Mamba-main"

/kaggle/input/ppp-2/pytorch/default/1/3DSS-Mamba-main/3DSS-Mamba-main


In [4]:
import torch
import os
from config import config
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpus

import numpy as np
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import torch
import torch.nn as nn
import torch.optim as optim
from operator import truediv
import time

from utils.utils import Logger, mkdirs
from models.videomamba import VisionMamba

def loadData():
    if config.data == 'PaviaU':
        data = sio.loadmat('/kaggle/input/pavia-university-hsi/PaviaU.mat')['paviaU']
        labels = sio.loadmat('/kaggle/input/pavia-university-hsi/PaviaU_gt.mat')['paviaU_gt']

    elif config.data == 'Indian':
        data = sio.loadmat('./data/Indian/Indian_pines_corrected.mat')['indian_pines_corrected']
        labels = sio.loadmat('./data/Indian/Indian_pines_gt.mat')['indian_pines_gt']

    elif config.data == 'Houston2018':
        data = sio.loadmat('./data/Houston2018/houstonU2018.mat')['houstonU']
        labels = sio.loadmat('./data/Houston2018/houstonU2018.mat')['houstonU_gt']

    elif config.data == 'Houston2013':
        data = sio.loadmat('./data/Houston2013/HustonU_IM.mat')['hustonu']
        labels = sio.loadmat('./data/Houston2013/HustonU_gt.mat')['hustonu_gt']

    return data, labels

def applyPCA(X, numComponents):
    # Principal component analysis on HSI data
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))

    return newX

def padWithZeros(X, margin=2):

    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X

    return newX

def createImageCubes(X, y, windowSize, removeZeroLabels = True):

    # padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1

    return patchesData, patchesLabels

def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=testRatio,random_state=randomState,stratify=y)

    return X_train, X_test, y_train, y_test

def create_data_loader():

    X, y = loadData()
    index = np.nonzero(y.reshape(y.shape[0]*y.shape[1]))
    index = index[0]
    # The proportion of test samples
    test_ratio = config.test_ratio
    # patch size
    patch_size = config.patch_size
    # The dimension after PCA
    pca_components = config.pca_components

    print('Hyperspectral data shape: ', X.shape)
    print('Label shape: ', y.shape)
    groundtruth = y

    print('\n... ... PCA tranformation ... ...')
    X_pca = applyPCA(X, numComponents=pca_components)
    print('Data shape after PCA: ', X_pca.shape)
    print('\n... ... create data cubes ... ...')
    X_pca, y = createImageCubes(X_pca, y, windowSize=patch_size)
    print('Data cube X shape: ', X_pca.shape)
    print('Data cube y shape: ', y.shape)

    print('\n... ... create train & test data ... ...')
    Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
    print('Xtrain shape: ', Xtrain.shape)
    print('Xtest  shape: ', Xtest.shape)

    X = X_pca.reshape(-1, patch_size, patch_size, pca_components, 1)
    Xtrain = Xtrain.reshape(-1, patch_size, patch_size, pca_components, 1)
    Xtest = Xtest.reshape(-1, patch_size, patch_size, pca_components, 1)
    print('before transpose: Xtrain shape: ', Xtrain.shape)
    print('before transpose: Xtest  shape: ', Xtest.shape)

    X = X.transpose(0, 4, 3, 1, 2)
    Xtrain = Xtrain.transpose(0, 4, 3, 1, 2)
    Xtest = Xtest.transpose(0, 4, 3, 1, 2)
    print('after transpose: Xtrain shape: ', Xtrain.shape)
    print('after transpose: Xtest  shape: ', Xtest.shape)

    # Create train loader and test loader
    X = TestDS(X, y)
    trainset = TrainDS(Xtrain, ytrain)
    testset = TestDS(Xtest, ytest)
    train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                               batch_size=config.BATCH_SIZE_TRAIN,
                                               shuffle=True,
                                               drop_last=True
                                               )
    test_loader = torch.utils.data.DataLoader(dataset=testset,
                                               batch_size=config.BATCH_SIZE_TRAIN,
                                               shuffle=False,
                                               num_workers=0,
                                               drop_last=False
                                              )
    all_data_loader = torch.utils.data.DataLoader(dataset=X,
                                               batch_size=config.BATCH_SIZE_TRAIN,
                                               shuffle=False,
                                               num_workers=0,
                                               drop_last=False
                                             )

    return train_loader, test_loader, y, index, all_data_loader, groundtruth

""" Training dataset"""

class TrainDS(torch.utils.data.Dataset):

    def __init__(self, Xtrain, ytrain):

        self.len = Xtrain.shape[0]
        self.x_data = torch.FloatTensor(Xtrain)
        self.y_data = torch.LongTensor(ytrain)

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index]
    def __len__(self):

        return self.len

""" Testing dataset"""

class TestDS(torch.utils.data.Dataset):

    def __init__(self, Xtest, ytest):

        self.len = Xtest.shape[0]
        self.x_data = torch.FloatTensor(Xtest)
        self.y_data = torch.LongTensor(ytest)

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index]

    def __len__(self):

        return self.len

def train(train_loader):
    net = VisionMamba(
        group_type = config.group_type,
        k_group=config.k_group,
        embed_dim=config.embed_dim,
        dt_rank=config.dt_rank,  # Rank of the dynamic routing tensor
        d_inner=config.dim_inner,  # 256,  # Inner dimension of the model
        d_state=config.d_state,  # 256,  # State dimension of the model
        num_classes=config.num_classes,  # Number of output classes
        depth=config.depth,
        scan_type=config.scan_type,
        pos=config.pos,
        cls=config.cls,
        conv3D_channel=config.conv3D_channel,
        conv3D_kernel=config.conv3D_kernel,
        dim_patch=config.dim_patch,
        dim_linear=config.dim_linear,
    ).cuda()
    # Use cross entropyloss function
    criterion = nn.CrossEntropyLoss()
    # Initializes the optimizer
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    # Start training
    total_loss = 0
    for epoch in range(config.train_epoch):
        net.train()
        for i, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()  #[64, 1, 30, 15, 15]
            outputs, _ = net(data)   #[64, 9]
            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        log.write('[Epoch: %d]   [loss avg: %.4f] \n' % (epoch + 1, total_loss / (epoch + 1)))
    log.write('Finished Training')
    return net

def mytest(net, test_loader):
    count = 0
    net.eval()
    y_pred_test = 0
    y_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.cuda()
            outputs, _ = net(inputs)
            outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
            if count == 0:
                y_pred_test = outputs
                y_test = labels
                count = 1
            else:
                y_pred_test = np.concatenate((y_pred_test, outputs))
                y_test = np.concatenate((y_test, labels))

    return y_pred_test, y_test

def AA_andEachClassAccuracy(confusion_matrix):

    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

def acc_reports(y_test, y_pred_test):

    oa = accuracy_score(y_test, y_pred_test)
    confusion = confusion_matrix(y_test, y_pred_test)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred_test)

    return oa*100, confusion, each_acc*100, aa*100, kappa*100

if __name__ == '__main__':
    mkdirs(config.checkpoint_path, config.checkpoint_path, config.logs)
    log = Logger()
    log.open(config.logs + config.data + '_log.txt', mode='a')

    oa = []
    acc = []
    aa = []
    kappa = []
    for num in range(config.test_epoch):
        train_loader, test_loader, y_all, index, all_data_loader, y = create_data_loader()
        net = train(train_loader)
        # Save model parameters
        # torch.save(net.state_dict(), 'LSFAT_params.pth')
        tic2 = time.perf_counter()
        y_pred_test, y_test = mytest(net, test_loader)  #(42776,)
        toc2 = time.perf_counter()
        # Evaluation indexes
        print("测试时间: {:.3f} 秒".format(toc2 - tic2))
        each_oa, confusion, each_acc, each_aa, each_kappa = acc_reports(y_test, y_pred_test)
        oa.append(each_oa)
        acc.append(each_acc)
        aa.append(each_aa)
        kappa.append(each_kappa)
        log.write('Test_Epoch: %.f: Each_OA: %.2f, Each_AA: %.2f, Each_kappa: %.2f \n' % (num+1, each_oa, each_aa, each_kappa))
    log.write('   AVG:   OA: %.2f, var: %.2f    AA: %.2f, var: %.2f    Kappa: %.2f, var: %.2f \n' % (np.mean(oa), np.var(oa), np.mean(aa), np.var(aa), np.mean(kappa), np.var(kappa)))

No module named 'selective_scan_cuda_oflex'
No module named 'selective_scan_cuda_core'


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/kaggle/input/ppp-2/pytorch/default/1/3DSS-Mamba-main/3DSS-Mamba-main/models/csms6s.py:231: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/kaggle/input/ppp-2/pytorch/default/1/3DSS-Mamba-main/3DSS-Mamba-main/models/csms6s.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/kaggle/input/ppp-2/pytorch/default/1/3DSS-Mamba-main/3DSS-Mamba-main/models/csms6s.py:254: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., devi

Hyperspectral data shape:  (610, 340, 103)
Label shape:  (610, 340)

... ... PCA tranformation ... ...
Data shape after PCA:  (610, 340, 30)

... ... create data cubes ... ...
Data cube X shape:  (42776, 11, 11, 30)
Data cube y shape:  (42776,)

... ... create train & test data ... ...
Xtrain shape:  (4277, 11, 11, 30)
Xtest  shape:  (38499, 11, 11, 30)
before transpose: Xtrain shape:  (4277, 11, 11, 30, 1)
before transpose: Xtest  shape:  (38499, 11, 11, 30, 1)
after transpose: Xtrain shape:  (4277, 1, 30, 11, 11)
after transpose: Xtest  shape:  (38499, 1, 30, 11, 11)
[Epoch: 1]   [loss avg: 117.3623] 
[Epoch: 2]   [loss avg: 103.0208] 
[Epoch: 3]   [loss avg: 89.9472] 
[Epoch: 4]   [loss avg: 80.2868] 
[Epoch: 5]   [loss avg: 72.6375] 
[Epoch: 6]   [loss avg: 66.3536] 
[Epoch: 7]   [loss avg: 61.2598] 
[Epoch: 8]   [loss avg: 56.8799] 
[Epoch: 9]   [loss avg: 53.2744] 
[Epoch: 10]   [loss avg: 50.0926] 
[Epoch: 11]   [loss avg: 47.2082] 
[Epoch: 12]   [loss avg: 44.6742] 
[Epoch: 13]